<a href="https://colab.research.google.com/github/OliverWills/Yachts/blob/main/Yacht_Model_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel
from google.colab import drive

In [ ]:
class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.2, freeze_bert=False):

        super(BertRegressor, self).__init__()
        D_in, D_out = 768, 1

        self.bert = \
                   BertModel.from_pretrained('bert-base-cased')
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
    def forward(self, input_ids, attention_masks):

        outputs = self.bert(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs

model = BertRegressor(drop_rate=0.2)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = f"/content/drive/My Drive/BERT_yacht"
#model.load_state_dict(torch.load(path,map_location=torch.device('cpu')))

In [ ]:
link = 'https://rss.apolloduck.com/gb2.rss'

In [ ]:
!pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=d17cd5934c941711f6a803ae3ab33d7ddd45b3b0f736c81c4055472c29f153ee
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
import feedparser

In [ ]:
feed = feedparser.parse(link)

In [ ]:
feed.entries[0].keys()

dict_keys(['id', 'title', 'title_detail', 'links', 'link', 'summary', 'summary_detail'])

In [ ]:
feed_dict = {'id':[], 'title':[], 'summary':[]}

In [ ]:
import numpy as np

In [ ]:
for entry in feed.entries:
  feed_dict['id'].append(entry.id if entry.id else np.nan)
  feed_dict['title'].append(entry.title if entry.title else np.nan)
  feed_dict['summary'].append(entry.summary if entry.summary else np.nan)


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(feed_dict)

In [ ]:
print(df['id'][0])

https://yachts.apolloduck.co.uk/boat.phtml?id=728986


In [ ]:
print(df.columns)

Index(['id', 'title', 'summary'], dtype='object')


In [ ]:
df['combined']="This is "+df['Title']+" yacht. It is located in "+df['Location']+". Tax status is "+df['Tax Status']+". It was built in "+df['Year'].astype(str)+". It has a length of "+df['Length_combined'].astype(str)+" meters. "+df['Description']
descriptions=df['combined'].tolist()


KeyError: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
df['Description_Long'] = np.nan
df['Price'] = np.nan
df['Status'] = np.nan
df['Tax_status'] = np.nan
df['Location'] = np.nan
df['Year'] = np.nan
df['Length'] = np.nan
df['Description_Short'] = np.nan
df['Keywords'] = np.nan


In [ ]:
i=0
for id in df['id'].tolist():
  page = requests.get(id, headers=headers)
  soup = BeautifulSoup(page.content, "html.parser")
  print(id)
  print(soup.find('meta', attrs={"name": "Description"})['content'])
  df.loc[i,'Description_Long'] = soup.find('div', class_="_featureSection cp").get_text()
  df.loc[i,'Price'] = soup.find('div', class_="_boatAdvertPrice").get_text()
  df.loc[i,'Status'] = soup.find('td', class_="_pclLabel", text=re.compile('Status:')).parent.find_all('td')[1].get_text()
  df.loc[i,'Tax_status'] = soup.find('td', class_="_pclLabel", text=re.compile('Tax Status:')).parent.find_all('td')[1].get_text() if soup.find('td', class_="_pclLabel", text=re.compile('Tax Status:')) else ''
  df.loc[i,'Location'] = soup.find('td', class_="_pclLabel", text=re.compile('Location:')).parent.find_all('td')[1].get_text()
  df.loc[i,'Year'] = ''
  df.loc[i,'Length'] = soup.find('td', class_="featureSpecLabel", text=re.compile('Length over all:')).parent.find_all('td')[1].get_text() if soup.find('td', class_="featureSpecLabel", text=re.compile('Length over all:')) else ''
  df.loc[i, 'Description_Short'] = soup.find('meta', attrs={"name": "Description"})['content']
  df.loc[i, 'Keywords'] = soup.find('meta', attrs={"name": "Keywords"})['content']
  i=i+1

https://yachts.apolloduck.co.uk/boat.phtml?id=727846
Sailing Yachts: For Sale: Cornish Shrimper 19 1985 Mk1 Hampshire


<ipython-input-197-52e0465421a0>:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  df.loc[i,'Status'] = soup.find('td', class_="_pclLabel", text=re.compile('Status:')).parent.find_all('td')[1].get_text()
<ipython-input-197-52e0465421a0>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  df.loc[i,'Tax_status'] = soup.find('td', class_="_pclLabel", text=re.compile('Tax Status:')).parent.find_all('td')[1].get_text() if soup.find('td', class_="_pclLabel", text=re.compile('Tax Status:')) else ''
<ipython-input-197-52e0465421a0>:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  df.loc[i,'Location'] = soup.find('td', class_="_pclLabel", text=re.compile('Location:')).parent.find_all('td')[1].get_text()
<ipython-input-197-52e0465421a0>:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' in

https://yachts.apolloduck.co.uk/boat.phtml?id=727844
Sailing Yachts: For Sale: Stag 28 Waldringfield Boatyard Suffolk
https://yachts.apolloduck.co.uk/boat.phtml?id=725187
Sailing Yachts: For Sale: Westerly Fulmar  Lifting Keel model Ashore Hayling Island Hampshire
https://yachts.apolloduck.co.uk/boat.phtml?id=642843
Sailing Yachts: For Sale: Nicholson 32  sold  Kent
https://yachts.apolloduck.co.uk/boat.phtml?id=725178
Sailing Yachts: For Sale: ARCADIA Surrey
https://yachts.apolloduck.co.uk/boat.phtml?id=716632
Sailing Yachts: For Sale: 2018 Jeanneau Sun Odyssey 440 Brighton and Hove
https://yachts.apolloduck.co.uk/boat.phtml?id=687715
Sailing Yachts: For Sale: Evolution 22 Nairn Highland
https://yachts.apolloduck.co.uk/boat.phtml?id=730709
Sailing Yachts: For Sale: 28 foot Taipan 28 Newell design Truro Cornwall
https://yachts.apolloduck.co.uk/boat.phtml?id=730708
Sailing Yachts: For Sale: Sadler 25
https://yachts.apolloduck.co.uk/boat.phtml?id=730684
Sailing Yachts: For Sale: 2004 Bene

In [ ]:
ad_link = 'https://yachts.apolloduck.co.uk/boat.phtml?id=716638'
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"}
page = requests.get(ad_link, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
import re

In [ ]:
soup.find('meta', name_='Desription', text=re.compile('content'))

NameError: ignored

In [ ]:
status = soup.find('td', class_="_pclLabel", text=re.compile('Location:')).string

<ipython-input-133-cc17f0d756b7>:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  status = soup.find('td', class_="_pclLabel", text=re.compile('Status:')).string


In [ ]:
print(status)

Status:


In [ ]:
soup.find('meta', attrs={"name": "Description"})['content']

'Sailing Yachts: For Sale: 1992 Mascot 35DS Ardrossan North Ayrshire'

In [ ]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <title>\n   Mascot 35 DS for sale UK, Mascot boats for sale, Mascot used boat sales, Mascot Sailing Yachts For Sale 1992 Mascot 35DS - Apollo Duck\n  </title>\n  <meta charset="utf-8"/>\n  <link as="style" href="/cache/apolloduck.min.css?220920011" rel="preload" type="text/css"/>\n  <link as="style" href="/cache/css/499.min.css?220920011" rel="preload" type="text/css"/>\n  <link as="style" href="/cache/css/699.min.css?220920011" rel="preload" type="text/css"/>\n  <link as="style" href="/cache/css/999.min.css?220920011" rel="preload" type="text/css"/>\n  <link as="style" href="/cache/css/999plus.min.css?220920011" rel="preload" type="text/css"/>\n  <link as="style" href="/cache/cookies/light-bottom.css" rel="preload" text="text/css">\n   <link as="script" href="/cache/js/jquery-3.5.1.min.min.js" rel="preload"/>\n   <link as="script" href="https://cache.apolloduck.com/jquery-ui-1.12.1.custom/jquery-ui.min.js" rel="preload"/>\n   <link as="st

In [ ]:
print(df.tail())

                                                   id  \
10  https://yachts.apolloduck.co.uk/boat.phtml?id=...   
11  https://yachts.apolloduck.co.uk/boat.phtml?id=...   
12  https://yachts.apolloduck.co.uk/boat.phtml?id=...   
13  https://yachts.apolloduck.co.uk/boat.phtml?id=...   
14  https://yachts.apolloduck.co.uk/boat.phtml?id=...   

                                   title  \
10     For Sale: 2005 Bavaria 30 Cruiser   
11                       For Sale: UFO34   
12     For Sale: Heavy Duty Motor Sailor   
13    For Sale: C  amp  C 40   C amp C40   
14  For Sale: 45ft  HISTORIC LOOE LUGGER   

                                              summary  \
10  In my opinion  the Bavaria 30 yacht has to be ...   
11  Classic cruiser racer UFO34  Great condition w...   
12  30 rsquo   8 ton Ketch  Fully enclosed  6 bert...   
13  Manitou is a special cruiser racer  She was de...   
14  OUR DADDY was built in Looe  Cornwall in 1921 ...   

                                     Description_

In [ ]:
print(df['id'].tolist())

['https://yachts.apolloduck.co.uk/boat.phtml?id=727846', 'https://yachts.apolloduck.co.uk/boat.phtml?id=727844', 'https://yachts.apolloduck.co.uk/boat.phtml?id=725187', 'https://yachts.apolloduck.co.uk/boat.phtml?id=642843', 'https://yachts.apolloduck.co.uk/boat.phtml?id=725178', 'https://yachts.apolloduck.co.uk/boat.phtml?id=716632', 'https://yachts.apolloduck.co.uk/boat.phtml?id=687715', 'https://yachts.apolloduck.co.uk/boat.phtml?id=730709', 'https://yachts.apolloduck.co.uk/boat.phtml?id=730708', 'https://yachts.apolloduck.co.uk/boat.phtml?id=730684', 'https://yachts.apolloduck.co.uk/boat.phtml?id=730682', 'https://yachts.apolloduck.co.uk/boat.phtml?id=728854', 'https://yachts.apolloduck.co.uk/boat.phtml?id=698002', 'https://yachts.apolloduck.co.uk/boat.phtml?id=687186', 'https://yachts.apolloduck.co.uk/boat.phtml?id=617310']


In [ ]:
descriptions=data['combined'].tolist()

NameError: ignored

In [ ]:
df.to_csv('yacht_rss.csv')

In [ ]:
data['combined']="This is "+data['Title']+" yacht. It is located in "+data['Location']+". Tax status is "+data['Tax Status']+". It was built in "+data['Year'].astype(str)+". It has a length of "+data['Length_combined'].astype(str)+" meters. "+data['Description']bb

In [ ]:
print(df['Keywords'].tolist)

In [ ]:
df['Keywords'] =